 02 – DATA PREPROCESSING

The purpose of this notebook is to clean and prepare the raw data so it’s suitable for analysis and modeling. This step includes loading the dataset, checking for issues like missing values or duplicates, and making sure the data types are correct. It also covers encoding categorical variables, scaling features, and preparing the train/test split.

1. Load the Datasets
   
Instead of loading a single file, this step loads all CSV files located in the data/raw/ folder into a dictionary of pandas DataFrames. Each dataset is stored using its filename (without the .csv extension) as the key. This approach allows easy access and inspection of multiple datasets simultaneously.

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [10]:
import os
import glob
import pandas as pd

# Path to the raw data folder
raw_data_path = '/content/drive/MyDrive/yacht-data-insights/data/raw/'

# Get a list of all CSV files in the folder
csv_files = glob.glob(os.path.join(raw_data_path, '*.csv'))

# Create a dictionary of DataFrames with the filename (without extension) as the key
dataframes = {}

for file in csv_files:
    name = os.path.splitext(os.path.basename(file))[0]  # filename without path or extension
    try:
        # Try reading the CSV with a specified encoding
        df = pd.read_csv(file, encoding='ISO-8859-1', low_memory=False)
        dataframes[name] = df
        print(f"Loaded {name} with shape {df.shape}")
    except Exception as e:
        print(f"Error loading {name}: {e}")

# Optionally, check the first file to confirm
if csv_files:
    first_file = csv_files[0]
    print(f"\nFirst file preview: {pd.read_csv(first_file, encoding='ISO-8859-1').head()}")
else:
    print("No CSV files found.")


Loaded named_anchorages_v1_20191205 with shape (166508, 10)
Loaded CVP_loitering_202411 with shape (684, 14)
Loaded named_anchorages_v1_20181108 with shape (119748, 7)
Loaded Weather-for-Boating-Activities with shape (1060, 6)
Loaded CVP_ports_202411 with shape (1410, 14)
Loaded boat_data with shape (9888, 10)
Loaded CVP_encounters_202411 with shape (348, 14)
Loaded sar_vessel_detections_pipev20231026_202410 with shape (268681, 10)
Loaded named_anchorages_v2_20201104 with shape (166515, 10)
Loaded boat_dataset with shape (10344, 38)
Loaded Boats_No_Price_dataset with shape (936, 26)
Loaded named_anchorages_v2_20221206 with shape (166482, 10)
Loaded sar_vessel_detections_pipev3_202411 with shape (248247, 10)
Loaded sar_vessel_detections_pipev3_202412 with shape (239081, 10)

First file preview:        s2id        lat        lon     label sublabel     label_source iso3  \
0  3e4e429b  26.914042  52.220320   SHARJAH      NaN  top_destination  IRN   
1  1a575de7  -7.715992  11.724560  BLOC

<ipython-input-10-f3543a93a66b>:27: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  print(f"\nFirst file preview: {pd.read_csv(first_file, encoding='ISO-8859-1').head()}")


2. Initial Checks: Data Types, Missing Values, and Duplicates
This step checks for missing values, incorrect data types, and duplicate records — common issues that need to be addressed before proceeding.

In [ ]:
# Overview of the dataset
df.info()

# Count missing values per column
df.isnull().sum()

# Check for duplicates
df.duplicated().sum()

3. Handle Missing Values
Dropped rows with missing values in the target column, since they can’t be used for training. Other missing numerical values were filled using the median to avoid skewing from outliers.

In [ ]:
# Drop rows where the target is missing
df = df.dropna(subset=['target_column'])

# Fill missing values in numerical columns
df['numerical_column'] = df['numerical_column'].fillna(df['numerical_column'].median())

4. Fix Data Types
Some columns were not in the correct format for processing. Converting them to the appropriate data types ensures compatibility with future transformations and modeling steps.

In [ ]:
# Convert values to numeric
df['numeric_column'] = pd.to_numeric(df['numeric_column'], errors='coerce')

5. Remove Duplicates
Removing duplicate rows to avoid bias and redundancy during analysis or training.

In [ ]:
# Drop duplicates
df = df.drop_duplicates()

6. Encode Categorical Variables
Categorical columns were one-hot encoded to convert them into a numerical format suitable for machine learning models.

In [ ]:
# One-hot encode selected categorical column
df = pd.get_dummies(df, columns=['categorical_column'])

7. Scale Numerical Features
Numerical features were scaled using StandardScaler to ensure all features contribute equally to model performance.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Apply standard scaling
scaler = StandardScaler()
df[['numerical_column1', 'numerical_column2']] = scaler.fit_transform(df[['numerical_column1', 'numerical_column2']])

8. Split Dataset into Train/Test
The cleaned and prepared dataset was split into training and testing sets to allow model validation on unseen data.

In [ ]:
from sklearn.model_selection import train_test_split

# Separate features and target
X = df.drop('target_column', axis=1)
y = df['target_column']

# Create training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
